# Week 3 – BSM Chooser Option Pricing Model

**Project:** Avalok Capital – Advanced Chooser Option Pricing  
**Reference:** Huang, Wang & Wan (2021), *Exploration of JPMorgan Chooser Option Pricing*

## Model Overview

A **simple European chooser option** gives the holder the right, at decision date $T_1$, to choose whether the option becomes a European **call** or **put** with the same strike $K$ and maturity $T_2$.

### Two-Period GBM Simulation

**Period 1** ($0 \to T_1$):
$$S_{T_1} = S_0 \exp\!\left[\left(r - q - \tfrac{\sigma^2}{2}\right)T_1 + \sigma\sqrt{T_1}\,Z_1\right], \quad Z_1 \sim N(0,1)$$

**Decision at $T_1$:** Choose **call** if $S_{T_1} > K$, else **put**.

**Period 2** ($T_1 \to T_2$):
$$S_{T_2} = S_{T_1} \exp\!\left[\left(r - q - \tfrac{\sigma^2}{2}\right)(T_2 - T_1) + \sigma\sqrt{T_2 - T_1}\,Z_2\right]$$

**Payoff at $T_2$:**  Call → $\max(S_{T_2} - K, 0)$;  Put → $\max(K - S_{T_2}, 0)$

**Chooser price** = $e^{-rT_2} \cdot \mathbb{E}[\text{Payoff}]$

In [ ]:
import sys
from pathlib import Path
import yaml
import numpy as np
import pandas as pd

# Project root for imports
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.models.bsm_chooser import (
    simulate_gbm_paths,
    chooser_payoffs,
    price_chooser_mc,
    bsm_call,
    bsm_put,
    rubinstein_chooser,
)

# Load parameters
with open(PROJECT_ROOT / "config" / "model_params.yaml") as f:
    cfg = yaml.safe_load(f)

params = cfg["model"]
sim_cfg = cfg["simulation"]

print("Paper Parameters (Table 2):")
for k, v in params.items():
    print(f"  {k:>8s} = {v}")
print(f"\nSimulation: {sim_cfg['n_paths']} paths, seed={sim_cfg['seed']}")

## 1. GBM Path Simulation

Demonstrate single-period GBM: simulate $S_{T_1}$ from $S_0$.

In [ ]:
S0, K = params["s0"], params["k"]
r, q, sigma = params["r"], params["q"], params["sigma"]
T1, T2 = params["t1"], params["t2"]

# Simulate 10 paths for Period 1 (illustrative)
s_t1_demo = simulate_gbm_paths(S0, r, q, sigma, T1, n_paths=10, seed=0)

print("Period 1: S0 -> S_T1 (10 sample paths)")
print(f"  S0 = ${S0:.2f}")
print(f"  S_T1 values: {np.round(s_t1_demo, 2)}")
print(f"  Mean = ${s_t1_demo.mean():.2f}, Std = ${s_t1_demo.std():.2f}")

## 2. Chooser Decision & Payoff

At $T_1$: if $S_{T_1} > K$ → call; else → put.  
At $T_2$: compute payoff based on choice.

In [ ]:
# Full 2-period simulation for 10 paths (like paper's Table 3)
rng = np.random.default_rng(0)
z1 = rng.standard_normal(10)
z2 = rng.standard_normal(10)

s_t1_10 = simulate_gbm_paths(S0, r, q, sigma, T1, n_paths=10, z=z1)
s_t2_10 = simulate_gbm_paths(s_t1_10, r, q, sigma, T2 - T1, n_paths=10, z=z2)
choices_10, payoffs_10 = chooser_payoffs(s_t1_10, s_t2_10, K)

table = pd.DataFrame({
    "S_T1 ($)": np.round(s_t1_10, 2),
    "Choice": np.where(choices_10, "CALL", "PUT"),
    "S_T2 ($)": np.round(s_t2_10, 2),
    "Payoff ($)": np.round(payoffs_10, 2),
})
table.index = range(1, 11)
table.index.name = "Path"
print("Sample paths (10 simulations):\n")
print(table.to_string())

## 3. Monte Carlo Pricing (N = 10,000)

Run full MC with paper parameters. Report discounted chooser price and statistics.

In [ ]:
result = price_chooser_mc(
    s0=S0, k=K, r=r, q=q, sigma=sigma,
    t1=T1, t2=T2,
    n_paths=sim_cfg["n_paths"],
    seed=sim_cfg["seed"],
    use_proper_rule=False,  # Paper's simplified rule: S_T1 > K
)

print("Monte Carlo Chooser Option Pricing")
print("=" * 45)
print(f"  Paths:          {sim_cfg['n_paths']:,}")
print(f"  Mean payoff:    ${result['mean_payoff']:.4f}")
print(f"  Std payoff:     ${result['std_payoff']:.4f}")
print(f"  Discount:       {result['discount']:.6f}")
print(f"  Chooser price:  ${result['price']:.4f}")
print(f"  Std error:      ${result['se']:.4f}")
print(f"  95% CI:         [${result['price'] - 1.96*result['se']:.4f}, ${result['price'] + 1.96*result['se']:.4f}]")
print(f"  Call ratio:     {result['call_ratio']:.2%}")
print(f"  Put ratio:      {1 - result['call_ratio']:.2%}")

## 4. Analytic Rubinstein (1991) Formula

Closed-form simple chooser option price for cross-validation:

$$V = \underbrace{S e^{-qT_2} N(d_1) - K e^{-rT_2} N(d_2)}_{\text{Call}(S, K, T_2)} \;-\; S e^{-qT_2} N(-y_1) + K e^{-rT_2} N(-y_2)$$

where $d_1, d_2$ use $T_2$ and $y_1, y_2$ use mixed $T_1/T_2$ (see `bsm_chooser.py`).

In [ ]:
analytic_price = rubinstein_chooser(S0, K, r, q, sigma, T1, T2)
call_price = bsm_call(S0, K, r, q, sigma, T2)
put_price = bsm_put(S0, K, r, q, sigma, T2)

print("Analytic Pricing (Rubinstein 1991)")
print("=" * 45)
print(f"  BSM Call (T2):   ${call_price:.4f}")
print(f"  BSM Put  (T2):   ${put_price:.4f}")
print(f"  Call + Put:      ${call_price + put_price:.4f}")
print(f"  Chooser price:   ${analytic_price:.4f}")
print()
print("Comparison with Monte Carlo")
print("-" * 45)
print(f"  MC price:        ${result['price']:.4f}")
print(f"  Analytic price:  ${analytic_price:.4f}")
print(f"  Difference:      ${abs(result['price'] - analytic_price):.4f}")
print(f"  Rel. error:      {abs(result['price'] - analytic_price) / analytic_price:.4%}")
print()
print("Note: The chooser is always worth more than a plain call or put,")
print("but less than a straddle (call + put), because the holder must")
print("decide at T1, not at T2.")

## 5. Summary

| Method | Chooser Price |
|--------|---------------|
| Monte Carlo (N=10,000) | See output above |
| Rubinstein analytic | See output above |

The MC price converges to the analytic solution within standard error. Both the simplified decision rule ($S_{T_1} > K$) and the proper BSM-value comparison yield similar prices for these parameters because $r \approx q$.

**Next:** See `week3_validation.ipynb` for Table 3 reproduction and sensitivity analysis.